In [ ]:
import sys, os, numpy as np, pandas as pd
from google.colab import drive

from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

drive.mount('/content/drive')

project_path = '/content/drive/My Drive/Colab Notebooks/CIS 5200 Final Project/code'
if project_path not in sys.path:
    sys.path.append(project_path)

import importlib
import flight_delay_pipeline_simple
importlib.reload(flight_delay_pipeline_simple)
from flight_delay_pipeline_simple import get_preprocessing_pipeline
from sklearn.linear_model import Ridge

print("Loaded flight_delay_pipeline and preprocessing.")

DATA_FILE_PATH = "/content/drive/My Drive/Colab Notebooks/CIS 5200 Final Project/train_test_datasets/"

X_train_path = DATA_FILE_PATH + f'X_train_added_delay_20251202_034422.csv'
y_train_path = DATA_FILE_PATH + f'y_train_added_delay_20251202_034422.csv'
X_test_path  = DATA_FILE_PATH + f'X_test_added_delay_20251202_034422.csv'
y_test_path  = DATA_FILE_PATH + f'y_test_added_delay_20251202_034422.csv'

TARGET = "DEP_ADDED_DELAY"
X_train = pd.read_csv(X_train_path)
y_train = pd.read_csv(y_train_path)[TARGET].squeeze()
X_test  = pd.read_csv(X_test_path)
y_test  = pd.read_csv(y_test_path)[TARGET].squeeze()

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")



Mounted at /content/drive
Loaded flight_delay_pipeline and preprocessing.
Train shape: (5086383, 49), Test shape: (1271596, 49)


In [ ]:
import gc
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline

N = 150_000
X_sample = pd.read_csv(X_train_path, nrows=N)
y_sample = pd.read_csv(y_train_path, nrows=N)["DEP_ADDED_DELAY"]

prep = get_preprocessing_pipeline()


ridge = Ridge()

pipe = Pipeline([
    ("prep", prep),
    ("model", ridge)
])

param_dist = {"model__alpha": np.logspace(-3, 3, 12)}

search = RandomizedSearchCV(
    pipe,
    param_distributions=param_dist,
    n_iter=5,
    cv=2,
    n_jobs=-1,
    scoring="neg_mean_absolute_error",
    verbose=2
)

search.fit(X_sample, y_sample)

print("Best params:", search.best_params_)
x = search.best_params_
del X_sample, y_sample, search
gc.collect()

Fitting 2 folds for each of 5 candidates, totalling 10 fits
Best params: {'model__alpha': np.float64(1000.0)}


68

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
prep = get_preprocessing_pipeline()

alpha = x['model__alpha']

ridge_pipe = Pipeline(steps=[
    ("prep", prep),
    ("model", Ridge(alpha=alpha))
])
ridge_pipe.fit(X_train, y_train)
y_pred = ridge_pipe.predict(X_test)

mae  = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print("\Single-Stage Ridge + Preprocessing on Test Set (DEP_ADDED_DELAY, all points)")
print(f"MAE : {mae:.4f}")
print(f"MSE: {rmse:.4f}")
print(f"R^2 : {r2:.4f}")

<>:19: SyntaxWarning: invalid escape sequence '\S'
<>:19: SyntaxWarning: invalid escape sequence '\S'
/tmp/ipython-input-1218042118.py:19: SyntaxWarning: invalid escape sequence '\S'
  print("\Single-Stage Ridge + Preprocessing on Test Set (DEP_ADDED_DELAY, all points)")
/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


\Single-Stage Ridge + Preprocessing on Test Set (DEP_ADDED_DELAY, all points)
MAE : 8.7875
MSE: 271.5596
R^2 : 0.0435


In [ ]:
from sklearn.metrics import balanced_accuracy_score
y_pred_labels = ridge_pipe.predict(X_test)
bal_acc = balanced_accuracy_score(y_test_binary, y_pred_labels)
print("Balanced Accuracy:", bal_acc)

/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


NameError: name 'y_test_binary' is not defined

In [ ]:
# pip install
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 7.2 MB/s eta 0:00:00
